In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import os

In [5]:
for dirname, _, filenames in os.walk('C:/Users/Ballu-PC/Documents/sumit/course 6/Option 2'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

C:/Users/Ballu-PC/Documents/sumit/course 6/Option 2\Data+Dictionary (1).xlsx
C:/Users/Ballu-PC/Documents/sumit/course 6/Option 2\Open_Invoice_data (1).csv
C:/Users/Ballu-PC/Documents/sumit/course 6/Option 2\Received_Payments_Data (1).csv
C:/Users/Ballu-PC/Documents/sumit/course 6/Option 2\TElwcom churn cash.ipynb


In [6]:
customer = pd.read_csv("C:/Users/Ballu-PC/Documents/sumit/course 6/Option 2/Received_Payments_Data (1).csv")
prod_info = pd.read_csv("C:/Users/Ballu-PC/Documents/sumit/course 6/Option 2/Open_Invoice_data (1).csv")
transaction = pd.read_csv("C:/Users/Ballu-PC/Documents/sumit/course 6/Option 2/Data+Dictionary (1).xlsx")

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xa0 in position 161787: invalid start byte

Exploratory Data Analysis

In [ ]:
customer.shape

In [ ]:
prod_info.shape

In [ ]:
transaction.shape

In [ ]:
customer.head(2)

In [ ]:
prod_info.head(2)


In [ ]:
# renaming "prod_sub_cat_code" column in 'prod_info' table to make it similar to 'transaction' table
# to merge the both the tables easily
prod_info.rename(columns={"prod_sub_cat_code":"prod_subcat_code"},inplace=True)

In [ ]:
transaction.head()

Merge 'transaction' and 'prod_info' tables

In [ ]:
# merge transaction and prod_info table and create a new table "prod_concat"
prod_concat = pd.merge(left=transaction, right=prod_info,on=["prod_cat_code","prod_subcat_code"],how="left")

In [ ]:
prod_concat


In [ ]:
prod_concat.isnull().sum()


Merge 'customer' and 'prod_concat' tables

In [ ]:
customer.head()


In [ ]:
#merge "prod_concat" and "customer" table and create the final table "customer_final"
customer_final = pd.merge(left=prod_concat, right=customer,right_on="customer_Id", left_on="cust_id", how="left")

In [ ]:
customer_final.head()

In [ ]:
customer_final.shape

In [ ]:
transaction.shape

In [ ]:
print('''Rows of both the 'customer_final' and 'transaction' table are same. That means all the transactions done at the 
         Retail Store are present in the final table ''')

In [ ]:
customer_final.dtypes

In [ ]:
customer_final.isnull().sum()

In [ ]:
# converting "DOB" and "tran_date" from object dtype to dates
customer_final["DOB"] = pd.to_datetime(customer_final["DOB"], format="%d-%m-%Y")

In [ ]:
customer_final['DOB'].head(10)

In [ ]:
customer_final["tran_date"] = pd.to_datetime(customer_final["tran_date"])

In [ ]:
customer_final["tran_date"].head(10)

Checking for duplicate values

In [ ]:
customer_final.duplicated().sum()

In [ ]:
# dropping duplicate rows
customer_final.drop_duplicates(inplace=True)

In [ ]:
customer_final.duplicated().sum()

2. Prepare a summary report for the merged data set.

(a) Get the column names and their corresponding data types

In [ ]:
#column names of "customer_final" dataframe
customer_final.columns

In [ ]:
# data types of all columns of "customer_final" dataframe
customer_final.dtypes

(b) Top/Bottom 10 observations

In [ ]:
# top 10 observations
customer_final.head(10)


In [ ]:
#bottom 10 observations
customer_final.tail(10)

(c) “Five-number summary” for continuous variables (min, Q1, median, Q3 and max)

In [ ]:
customer_final.describe()


(d) Frequency tables for all the categorical variables

In [ ]:
customer_final.loc[:,customer_final.dtypes=="object"].describe()


(3) Generate histograms for all continuous variables and frequency bars for categorical variables


Histogram of all continuous variables

In [ ]:
conti_customer = customer_final.loc[:,['prod_subcat_code','prod_cat_code', 'Qty', 'Rate', 'Tax', 'total_amt']]

In [ ]:
conti_customer.columns

In [ ]:
for var in conti_customer.columns:
    conti_customer[var].plot(kind='hist')
    plt.title(var)
    plt.show()

In [ ]:
category_customer = customer_final.loc[:,customer_final.dtypes=='object']

In [ ]:
category_customer.head()

In [ ]:
plt.figure(figsize=(8,8))
sns.countplot(category_customer['Gender'])
plt.show()

In [ ]:
plt.figure(figsize=(8,8))
sns.countplot(category_customer['Store_type'])
plt.xlabel('Store Type')
plt.show()

In [ ]:
plt.figure(figsize=(8,8))
sns.countplot(category_customer['prod_cat'])
plt.xlabel('Product Category')
plt.show()

In [ ]:
plt.figure(figsize=(8,8))
category_customer.groupby('prod_subcat')['prod_subcat'].count().plot(kind='barh')
plt.xlabel('Count')
plt.ylabel('Product Subcategory')
plt.show()

(4) Calculate the following information using the merged dataset:

(a) Time period of the available transaction data

In [2]:
customer_final.sort_values(by="tran_date")

NameError: name 'customer_final' is not defined

In [ ]:
min_date = customer_final["tran_date"].min()


In [ ]:
max_date = customer_final["tran_date"].max()


In [ ]:
print("Time period of the available transaction data is from "+ pd.Timestamp.strftime(min_date,format="%d-%m-%Y") + " to " + pd.Timestamp.strftime(max_date,format="%d-%m-%Y"))

(b) Count of transactions where the total amount of transaction was negative

In [ ]:
customer_final.head()


In [ ]:
#count of transaction_ids where total_amt was negative
negative_transaction = customer_final.loc[customer_final["total_amt"] < 0,"transaction_id"].count()

In [ ]:
print("Count of transactions where the total amount of transaction was negative is",negative_transaction)

(5) Analyze which product categories are more popular among females vs male customers

In [ ]:
#groupby the data set on the basis of "Gender" and "prod_cat"
product_gender = customer_final.groupby(["Gender","prod_cat"])[["Qty"]].sum().reset_index()

In [ ]:
product_gender

In [ ]:
#converting to pivot table for better view
product_gender.pivot(index="Gender",columns="prod_cat",values="Qty")

(6) Which City code has the maximum customers and what was the percentage of customers from that city?

In [ ]:
customer_final.head(2)


In [ ]:
customer_group = customer_final.groupby('city_code')['customer_Id'].count().sort_values(ascending =False)

In [ ]:
customer_group

In [ ]:
plt.figure(figsize=(8,5))
customer_group.plot(kind="bar")
plt.xlabel("City Code")
plt.ylabel("No. of customers")
plt.yticks(np.arange(0, 3500, step=500))
plt.show()

In [ ]:
percentage = round((customer_group[4.0] / customer_group.sum()) * 100,2)

In [ ]:
percentage

In [ ]:
print("City code 4.0 has the maximum customers and the percentage of customers from that city is ",percentage)

(7) Which store type sells the maximum products by value and by quantity?

In [ ]:
customer_final.head(2)


In [ ]:
customer_final.groupby("Store_type")["Qty","Rate"].sum().sort_values(by="Qty",ascending=False)

In [ ]:
print('e-Shop store sell the maximum products by value and by quantity')

(8) What was the total amount earned from the "Electronics" and "Clothing" categories from Flagship Stores?

In [ ]:
store_group = round(customer_final.pivot_table(index = "prod_cat",columns="Store_type", values="total_amt", aggfunc='sum'),2)

In [ ]:
store_group

In [ ]:
store_group.loc[["Clothing","Electronics"],"Flagship store"]

In [ ]:
# if we have to find total amount of both 'Clothing' and 'Electronics' from ' Flagship Store'
store_group.loc[["Clothing","Electronics"],"Flagship store"].sum()

(9) What was the total amount earned from "Male" customers under the "Electronics" category?

In [ ]:
gender_group = round(customer_final.pivot_table(index = "prod_cat",columns="Gender", values="total_amt", aggfunc='sum'),2)

In [ ]:
gender_group

In [ ]:
male_earning = gender_group.loc["Electronics","M"]


In [ ]:
print("The total amount earned from Male customers under the Electronics category is",male_earning)

(10) How many customers have more than 10 unique transactions, after removing all transactions which have any negative amounts?

In [ ]:
#creating a new dataframe that does not contain transactions with negative values
pos_trans = customer_final.loc[customer_final["total_amt"]>0,:]

In [ ]:
pos_trans

In [ ]:
# creating a dataframe that contains unique transactions 
unique_trans = pos_trans.groupby(['customer_Id','prod_cat','prod_subcat'])['transaction_id'].count().reset_index()

In [ ]:
unique_trans

In [ ]:
# now finding the customers which have unique transactions greater than 10
unique_trans_count = unique_trans.groupby('customer_Id')['transaction_id'].count().reset_index()

In [ ]:
unique_trans_count.head()


In [ ]:
unique_trans_count[unique_trans_count['transaction_id'] > 10]


In [ ]:
print('There are no unique transactions greater than 10')


(11) For all customers aged between 25-35, find out:


(a) What was the total amount spent for 'Electronics' and 'Books' product categories?

In [ ]:
now = pd.Timestamp('now')
customer_final['DOB'] = pd.to_datetime(customer_final['DOB'], format='%m%d%y')    # 1
customer_final['DOB'] = customer_final['DOB'].where(customer_final['DOB'] < now, customer_final['DOB'] -  np.timedelta64(100, 'Y'))   # 2
customer_final['AGE'] = (now - customer_final['DOB']).astype('<m8[Y]')

In [3]:
customer_final.head()

NameError: name 'customer_final' is not defined

as we have to deal with customers aged between 25-35, so creating new column 'Age_cat'

In [ ]:
customer_final['Age_cat'] = pd.cut(customer_final['AGE'],bins=[24,35,46,57],labels=['25-35','36-46','47-57'],include_lowest=True)

In [ ]:
customer_final.head()


In [ ]:
# grouping the dataframe 'customer_final' on the basis of 'Age_cat' and 'prod_cat'
customer_25_35 = customer_final.groupby(['Age_cat','prod_cat'])['total_amt'].sum()

In [ ]:
customer_25_35


In [ ]:
customer_25_35.loc['25-35',['Books','Electronics']]


In [ ]:
print("Total amount spent on 'Electronics' and 'Books' product categories is", 
      customer_25_35.loc['25-35',['Books','Electronics']].sum().round(2))

(b) What was the total amount spent by these customers between 1st Jan 2014 to 1st Mar 2014?

In [ ]:
customer_final.head()


In [ ]:
# filtering out data that belongs to the 'age_cat' = 25-35
customer_total_amount_25_35 = customer_final[customer_final['Age_cat']=='25-35']

In [ ]:
customer_total_amount_25_35.head()


In [ ]:
# getting all the data with transaction date between 1st Jan 2014 to 1st Mar 2014?
total_amount = customer_total_amount_25_35[(customer_total_amount_25_35['tran_date'] >='2014-01-01') & (customer_total_amount_25_35['tran_date'] <='2014-03-01')]

In [ ]:
total_amount

In [ ]:
print('The total amount spent by customers aged 25-35 between 1st Jan 2014 to 1st Mar 2014 is',
      total_amount['total_amt'].sum())